In [23]:
import pandas as pd
import datetime
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import model_selection

In [8]:
data = pd.read_csv("dacon_competition/modeling_data.csv", encoding = 'CP949')

In [9]:
data.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,month,year,deceased,festival_count,rain_day,covid,age_population,total_sale_by_indst_groupby,log_total_sale_by_indst,total_tourist_groupby,home_ratio
0,강원,건강보조식품 소매업,강원,20s,1,1,843100.0,11,2019,0,0,8.806154,False,15622761472137,42767.077,10.591,1.980289e+06,0.710718
1,강원,건강보조식품 소매업,강원,20s,1,1,139000.0,6,2019,0,8,11.461487,False,15622761472137,42767.077,10.591,1.980289e+06,0.710718
2,강원,건강보조식품 소매업,강원,20s,1,1,27500.0,8,2019,0,8,11.461487,False,15622761472137,42767.077,10.591,1.980289e+06,0.710718
3,강원,건강보조식품 소매업,강원,20s,1,1,395500.0,9,2019,0,11,11.461487,False,15622761472137,42767.077,10.591,1.980289e+06,0.710718
4,강원,건강보조식품 소매업,강원,20s,1,1,427510.0,3,2020,1,0,7.066787,True,15622761472137,42767.077,10.591,NaN,0.710718


In [10]:
del data['deceased']

In [14]:
data = pd.get_dummies(data, columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'month', 'year', 'covid'])

In [56]:
data.head()

,AMT,festival_count,rain_day,age_population,total_sale_by_indst_groupby,log_total_sale_by_indst,total_tourist_groupby,home_ratio,CARD_SIDO_NM_강원,CARD_SIDO_NM_경기,...,month_7,month_8,month_9,month_10,month_11,month_12,year_2019,year_2020,covid_False,covid_True
0,843100.0,0,8.806154,15622761472137,42767.077,10.591,1.980289e+06,0.710718,1,0,...,0,0,0,0,1,0,1,0,1,0
1,139000.0,8,11.461487,15622761472137,42767.077,10.591,1.980289e+06,0.710718,1,0,...,0,0,0,0,0,0,1,0,1,0
2,27500.0,8,11.461487,15622761472137,42767.077,10.591,1.980289e+06,0.710718,1,0,...,0,1,0,0,0,0,1,0,1,0
3,395500.0,11,11.461487,15622761472137,42767.077,10.591,1.980289e+06,0.710718,1,0,...,0,0,1,0,0,0,1,0,1,0
4,427510.0,0,7.066787,15622761472137,42767.077,10.591,NaN,0.710718,1,0,...,0,0,0,0,0,0,0,1,0,1


In [92]:
data['age_population'] = np.log(data['age_population'])

In [94]:
del data['total_sale_by_indst_groupby']

In [95]:
data['total_tourist_groupby'] = np.log(data['total_tourist_groupby'])

## Train-test split

In [96]:
x_train, x_test, y_train, y_test = train_test_split(data.drop(columns = ['AMT']),data['AMT'], test_size = 0.3)

## Catboost Modeling

In [62]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from catboost import Pool, CatBoostRegressor
from sklearn.metrics import mean_squared_error

In [20]:
def rmsle(h, y): 
    """
    Compute the Root Mean Squared Log Error for hypthesis h and targets y

    Args:
        h - numpy array containing predictions with shape (n_samples, n_targets)
        y - numpy array containing targets with shape (n_samples, n_targets)
    """
    return np.sqrt(np.square(np.log(h + 1) - np.log(y + 1)).mean())

In [107]:
model = CatBoostRegressor(iterations = 1200, learning_rate = 0.03, depth = 2, loss_function = 'RMSE', random_seed = 2020, verbose=True, task_type = "GPU")

In [108]:
train_pool = Pool(x_train, 
                  y_train)
test_pool = Pool(x_test) 

In [109]:
model.fit(train_pool)

0:	learn: 471927766.7878512	total: 85.5ms	remaining: 1m 42s
1:	learn: 471129202.5782797	total: 146ms	remaining: 1m 27s
2:	learn: 470376757.7769100	total: 223ms	remaining: 1m 28s
3:	learn: 469667415.3229236	total: 300ms	remaining: 1m 29s
4:	learn: 468999191.2637396	total: 506ms	remaining: 2m
5:	learn: 468369611.4692206	total: 631ms	remaining: 2m 5s
6:	learn: 467776256.5339190	total: 675ms	remaining: 1m 55s
7:	learn: 467217337.0919370	total: 796ms	remaining: 1m 58s
8:	learn: 466691073.3869901	total: 895ms	remaining: 1m 58s
9:	learn: 466195017.9375415	total: 1.04s	remaining: 2m 3s
10:	learn: 465728013.0785306	total: 1.09s	remaining: 1m 58s
11:	learn: 465443930.1997548	total: 1.13s	remaining: 1m 52s
12:	learn: 465002393.8414652	total: 1.17s	remaining: 1m 46s
13:	learn: 464738710.8266656	total: 1.19s	remaining: 1m 41s
14:	learn: 464326198.3164033	total: 1.24s	remaining: 1m 37s
15:	learn: 464081456.8917598	total: 1.28s	remaining: 1m 34s
16:	learn: 463846713.1463675	total: 1.33s	remaining: 1m

138:	learn: 446951934.6887187	total: 9.5s	remaining: 1m 12s
139:	learn: 446933678.8332823	total: 9.58s	remaining: 1m 12s
140:	learn: 446916157.4116790	total: 9.63s	remaining: 1m 12s
141:	learn: 446894877.5586718	total: 9.71s	remaining: 1m 12s
142:	learn: 446879614.8008099	total: 9.76s	remaining: 1m 12s
143:	learn: 446862458.8931614	total: 9.8s	remaining: 1m 11s
144:	learn: 446846378.0356709	total: 9.88s	remaining: 1m 11s
145:	learn: 446819988.3803915	total: 9.92s	remaining: 1m 11s
146:	learn: 446789457.2267045	total: 10s	remaining: 1m 11s
147:	learn: 446643935.7622710	total: 10s	remaining: 1m 11s
148:	learn: 446472447.2513579	total: 10.1s	remaining: 1m 11s
149:	learn: 446446444.3554953	total: 10.1s	remaining: 1m 10s
150:	learn: 446417223.3613147	total: 10.2s	remaining: 1m 10s
151:	learn: 446280169.9113947	total: 10.2s	remaining: 1m 10s
152:	learn: 446258055.2964074	total: 10.2s	remaining: 1m 10s
153:	learn: 446242307.1682320	total: 10.3s	remaining: 1m 9s
154:	learn: 446218908.9229473	t

276:	learn: 442193327.7300464	total: 18s	remaining: 1m
277:	learn: 442178824.6277612	total: 18.1s	remaining: 1m
278:	learn: 442170678.4641497	total: 18.2s	remaining: 60s
279:	learn: 442149830.7572150	total: 18.2s	remaining: 59.8s
280:	learn: 442142151.9481449	total: 18.2s	remaining: 59.7s
281:	learn: 442134651.9073225	total: 18.3s	remaining: 59.5s
282:	learn: 442123449.7745669	total: 18.3s	remaining: 59.3s
283:	learn: 442043733.0364704	total: 18.4s	remaining: 59.3s
284:	learn: 442018364.3534692	total: 18.4s	remaining: 59.2s
285:	learn: 441941602.4870201	total: 18.5s	remaining: 59.2s
286:	learn: 441905736.2489517	total: 18.6s	remaining: 59s
287:	learn: 441858974.9697480	total: 18.7s	remaining: 59.1s
288:	learn: 441850299.6299510	total: 18.7s	remaining: 59s
289:	learn: 441839888.9973502	total: 18.8s	remaining: 58.9s
290:	learn: 441829726.0003445	total: 18.8s	remaining: 58.8s
291:	learn: 441819672.9413401	total: 18.9s	remaining: 58.9s
292:	learn: 441811809.3617750	total: 19s	remaining: 59

414:	learn: 439525061.2338803	total: 26.3s	remaining: 49.8s
415:	learn: 439520672.8647705	total: 26.4s	remaining: 49.8s
416:	learn: 439496459.9079586	total: 26.5s	remaining: 49.8s
417:	learn: 439486741.1247435	total: 26.6s	remaining: 49.8s
418:	learn: 439479721.8698685	total: 26.6s	remaining: 49.7s
419:	learn: 439474723.8705498	total: 26.7s	remaining: 49.5s
420:	learn: 439467829.0302753	total: 26.7s	remaining: 49.4s
421:	learn: 439460837.1641085	total: 26.7s	remaining: 49.3s
422:	learn: 439456669.6820549	total: 26.8s	remaining: 49.2s
423:	learn: 439444513.1464956	total: 26.9s	remaining: 49.1s
424:	learn: 439439736.2836556	total: 26.9s	remaining: 49.1s
425:	learn: 439432827.0481429	total: 27s	remaining: 49s
426:	learn: 439419119.0248510	total: 27s	remaining: 48.9s
427:	learn: 439406892.2149818	total: 27.1s	remaining: 48.8s
428:	learn: 439348509.0072496	total: 27.1s	remaining: 48.7s
429:	learn: 439341182.8624483	total: 27.2s	remaining: 48.7s
430:	learn: 439332208.5148104	total: 27.2s	rem

553:	learn: 438063943.2498895	total: 35.5s	remaining: 41.4s
554:	learn: 438029301.3681012	total: 35.6s	remaining: 41.3s
555:	learn: 438025273.0645051	total: 35.6s	remaining: 41.3s
556:	learn: 438021453.0872180	total: 35.7s	remaining: 41.2s
557:	learn: 438018813.8107234	total: 35.8s	remaining: 41.2s
558:	learn: 438012354.4616888	total: 35.9s	remaining: 41.1s
559:	learn: 438010256.8891479	total: 36s	remaining: 41.1s
560:	learn: 438007853.6976648	total: 36s	remaining: 41.1s
561:	learn: 437998143.5585014	total: 36.2s	remaining: 41.1s
562:	learn: 437986238.2729463	total: 36.4s	remaining: 41.2s
563:	learn: 437982265.1418045	total: 36.5s	remaining: 41.1s
564:	learn: 437978597.6041448	total: 36.6s	remaining: 41.1s
565:	learn: 437972679.4672498	total: 36.6s	remaining: 41s
566:	learn: 437958328.3478157	total: 36.7s	remaining: 41s
567:	learn: 437955021.8276994	total: 36.8s	remaining: 41s
568:	learn: 437947394.5075151	total: 36.9s	remaining: 40.9s
569:	learn: 437941531.5226477	total: 37s	remaining

691:	learn: 437122164.5968868	total: 45s	remaining: 33s
692:	learn: 437119018.7865455	total: 45.1s	remaining: 33s
693:	learn: 437096204.0243292	total: 45.1s	remaining: 32.9s
694:	learn: 437089842.4048556	total: 45.2s	remaining: 32.9s
695:	learn: 437086515.3941219	total: 45.3s	remaining: 32.8s
696:	learn: 437083049.1509819	total: 45.3s	remaining: 32.7s
697:	learn: 437080070.1088086	total: 45.3s	remaining: 32.6s
698:	learn: 437078260.4003783	total: 45.4s	remaining: 32.5s
699:	learn: 437073666.4914798	total: 45.5s	remaining: 32.5s
700:	learn: 437069239.5881305	total: 45.5s	remaining: 32.4s
701:	learn: 437066121.2393581	total: 45.6s	remaining: 32.4s
702:	learn: 437061429.7530841	total: 45.7s	remaining: 32.3s
703:	learn: 437059383.3026442	total: 45.7s	remaining: 32.2s
704:	learn: 437057420.3717902	total: 45.7s	remaining: 32.1s
705:	learn: 437052506.0452431	total: 45.8s	remaining: 32s
706:	learn: 437048273.8330535	total: 45.8s	remaining: 32s
707:	learn: 437038250.0091220	total: 45.9s	remaini

831:	learn: 436352171.5290068	total: 53.8s	remaining: 23.8s
832:	learn: 436350693.4596226	total: 53.9s	remaining: 23.7s
833:	learn: 436349201.4411098	total: 53.9s	remaining: 23.7s
834:	learn: 436337139.6087916	total: 54s	remaining: 23.6s
835:	learn: 436335047.9276453	total: 54s	remaining: 23.5s
836:	learn: 436325677.0730234	total: 54.1s	remaining: 23.5s
837:	learn: 436321981.6657954	total: 54.1s	remaining: 23.4s
838:	learn: 436320587.1643682	total: 54.2s	remaining: 23.3s
839:	learn: 436318146.7761459	total: 54.2s	remaining: 23.2s
840:	learn: 436309988.8078658	total: 54.3s	remaining: 23.2s
841:	learn: 436305791.2297103	total: 54.4s	remaining: 23.1s
842:	learn: 436304354.8398716	total: 54.5s	remaining: 23.1s
843:	learn: 436302918.4453040	total: 54.6s	remaining: 23s
844:	learn: 436294062.8986204	total: 54.6s	remaining: 22.9s
845:	learn: 436291566.5781822	total: 54.6s	remaining: 22.9s
846:	learn: 436285249.0106905	total: 54.7s	remaining: 22.8s
847:	learn: 436280325.9862904	total: 54.7s	rem

971:	learn: 435489130.9567476	total: 1m 1s	remaining: 14.3s
972:	learn: 435484506.3123769	total: 1m 1s	remaining: 14.3s
973:	learn: 435483123.1010450	total: 1m 1s	remaining: 14.2s
974:	learn: 435481837.6885961	total: 1m 1s	remaining: 14.2s
975:	learn: 435473915.5520359	total: 1m 1s	remaining: 14.1s
976:	learn: 435471526.3079613	total: 1m 1s	remaining: 14.1s
977:	learn: 435469388.5521514	total: 1m 1s	remaining: 14s
978:	learn: 435465182.8712006	total: 1m 1s	remaining: 13.9s
979:	learn: 435462821.5242239	total: 1m 1s	remaining: 13.9s
980:	learn: 435448443.5790088	total: 1m 1s	remaining: 13.8s
981:	learn: 435446892.5769589	total: 1m 1s	remaining: 13.7s
982:	learn: 435444978.2695145	total: 1m 1s	remaining: 13.7s
983:	learn: 435431689.6698047	total: 1m 2s	remaining: 13.6s
984:	learn: 435428559.5936166	total: 1m 2s	remaining: 13.6s
985:	learn: 435425177.9681635	total: 1m 2s	remaining: 13.5s
986:	learn: 435421698.4996850	total: 1m 2s	remaining: 13.4s
987:	learn: 435420496.7490919	total: 1m 2s

1107:	learn: 434868416.7031236	total: 1m 10s	remaining: 5.87s
1108:	learn: 434867269.3901085	total: 1m 10s	remaining: 5.8s
1109:	learn: 434864191.2182838	total: 1m 10s	remaining: 5.74s
1110:	learn: 434856229.8455558	total: 1m 10s	remaining: 5.68s
1111:	learn: 434848925.9624133	total: 1m 10s	remaining: 5.62s
1112:	learn: 434847428.7911394	total: 1m 11s	remaining: 5.56s
1113:	learn: 434846099.5226043	total: 1m 11s	remaining: 5.49s
1114:	learn: 434842937.2463892	total: 1m 11s	remaining: 5.43s
1115:	learn: 434831631.2212123	total: 1m 11s	remaining: 5.36s
1116:	learn: 434829798.1622088	total: 1m 11s	remaining: 5.3s
1117:	learn: 434828538.8042167	total: 1m 11s	remaining: 5.23s
1118:	learn: 434827349.4072086	total: 1m 11s	remaining: 5.17s
1119:	learn: 434826439.8661247	total: 1m 11s	remaining: 5.11s
1120:	learn: 434815763.1108495	total: 1m 11s	remaining: 5.04s
1121:	learn: 434799852.4399486	total: 1m 11s	remaining: 4.98s
1122:	learn: 434798159.1856372	total: 1m 11s	remaining: 4.91s
1123:	lear

In [110]:
preds = model.predict(test_pool)

In [111]:
np.min(preds)

-208018229.9489975

### 너무 음수가 나와서 실패..